In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/getdatachimp/demo/main/prep/borked_iris.csv')
df

In [ ]:
X = df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]
y = df['variety'].map({'Setosa': 0, 'Versicolor': 1, 'Virginica': 2})
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [3]:
pipeline = make_pipeline(
  StandardScaler(),
  LogisticRegression()
)

model = pipeline.fit(X_train, y_train)

In [4]:
model.score(X_test, y_test)

0.7368421052631579

In [5]:
from doubtlab.ensemble import DoubtEnsemble
from doubtlab.reason import ProbaReason, WrongPredictionReason
reasons = {
    'proba': ProbaReason(model=model),
    'wrong_pred': WrongPredictionReason(model=model)
}

# Pass these reasons to a doubtlab instance.
doubt = DoubtEnsemble(**reasons)

# Get the ordered indices of examples worth checking again
indices = doubt.get_indices(X_train, y_train)
# Get dataframe with "reason"-ing behind the sorting
predicates = doubt.get_predicates(X_train, y_train)

In [6]:
df = df.join(predicates).drop(['Unnamed: 0'], axis=1)
df.iloc[predicates.index].head(len(indices))

,sepal_length,sepal_width,petal_length,petal_width,variety,collector,predicate_proba,predicate_wrong_pred
99,5.7,2.8,4.1,1.3,Versicolor,Eric,1.0,1.0
55,5.7,2.8,4.5,1.3,Versicolor,Eric,1.0,1.0
92,5.8,2.6,4.0,1.2,Versicolor,Eric,1.0,1.0
41,4.5,2.3,1.3,0.3,Setosa,Sam,1.0,1.0
96,5.7,2.9,4.2,1.3,Versicolor,Eric,1.0,1.0
68,6.2,2.2,4.5,1.5,Versicolor,Eric,1.0,1.0
64,5.6,2.9,3.6,1.3,Versicolor,Eric,1.0,1.0
61,5.9,3.0,4.2,1.5,Versicolor,Eric,0.0,1.0
106,4.9,2.5,4.5,1.7,Virginica,Sam,0.0,1.0
87,6.3,2.3,4.4,1.3,Versicolor,Eric,0.0,1.0


Eric is always associated with doubtly doubtful predictions. What's up with that? Pairplot colored by collector could be helpful here.

In [ ]:
import seaborn as sns
sns.pairplot(data=df, hue='collector')

Notice how different the distributions are different by collector for petal_length and petal_width

In [29]:
# Use pandera to create simple check that features >= some values
import pandera as pa

schema = pa.DataFrameSchema({
  'variety': pa.Column(str),
  'petal_length': pa.Column(float, [
    pa.Check(lambda g: g['Versicolor'].max() <= 30, groupby='variety'), 
    pa.Check(lambda g: g['Setosa'] <= 10, groupby='variety'),
    pa.Check(lambda g: g['Virginica'] <= 40, groupby='variety')
    ])
})

schema.validate(df)


SchemaError: <Schema Column(name=petal_length, type=DataType(float64))> failed series or dataframe validator 0:
<Check <lambda>>

In [9]:
df.loc[df['collector'] == 'Eric', 'petal_length'] /= 100
df.loc[df['collector'] == 'Eric', 'petal_width'] /= 100

In [20]:
schema.validate(df)

,sepal_length,sepal_width,petal_length,petal_width,variety,collector,predicate_proba,predicate_wrong_pred
0,5.1,3.5,1.4,0.2,Setosa,Sam,0.0,0.0
1,4.9,3.0,1.4,0.2,Setosa,Sam,0.0,0.0
2,4.7,3.2,1.3,0.2,Setosa,Sam,0.0,0.0
3,4.6,3.1,1.5,0.2,Setosa,Sam,0.0,0.0
4,5.0,3.6,1.4,0.2,Setosa,Sam,0.0,0.0
...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Virginica,Sam,0.0,0.0
146,6.3,2.5,5.0,1.9,Virginica,Sam,0.0,0.0
147,6.5,3.0,5.2,2.0,Virginica,Sam,0.0,0.0
148,6.2,3.4,5.4,2.3,Virginica,Sam,0.0,0.0


In [48]:
# If there's time, use pandera for hypothesis testing check for something more robust

In [1]:
%connect_info

{
  "shell_port": 9007,
  "iopub_port": 9009,
  "stdin_port": 9008,
  "control_port": 9006,
  "hb_port": 9005,
  "ip": "127.0.0.1",
  "key": "0c4c47c1-669d-4105-99fd-83c515414e70",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-v2-645291MGUKVjOMOLK.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [47]:
# move to default.ipynb
import pandas as pd
min = pd.DataFrame(
    index=['Setosa', 'Versicolor', 'Virginica'], 
    data={
          'min_petal_length': [10, 30, 45],
          'min_petal_width': [1, 10, 14],
  })

result = (
    df[['variety', 'petal_length', 'petal_width']]
      .join(min, on='variety')
      .query('petal_length > min_petal_length | petal_width > min_petal_width')
)
result if not result.empty else None

,variety,petal_length,petal_width,min_petal_length,min_petal_width
0,Setosa,1.4,0.2,10,1
1,Setosa,1.4,0.2,10,1
2,Setosa,1.3,0.2,10,1
3,Setosa,1.5,0.2,10,1
4,Setosa,1.4,0.2,10,1
...,...,...,...,...,...
145,Virginica,5.2,2.3,45,14
146,Virginica,5.0,1.9,45,14
147,Virginica,5.2,2.0,45,14
148,Virginica,5.4,2.3,45,14


In [244]:
df.loc[df['collector'] == 'Eric', 'petal_length'] *= 100
df.loc[df['collector'] == 'Eric', 'petal_width'] *= 100

,sepal_length,sepal_width,petal_length,petal_width,variety,collector,predicate_proba,predicate_wrong_pred
0,5.1,3.5,1.400000,0.200000,Setosa,Sam,0.0,0.0
1,4.9,3.0,0.033635,0.004805,Setosa,Eric,0.0,0.0
2,4.7,3.2,1.300000,0.200000,Setosa,Sam,0.0,0.0
3,4.6,3.1,0.036038,0.004805,Setosa,Eric,0.0,0.0
4,5.0,3.6,0.033635,0.004805,Setosa,Eric,0.0,0.0
...,...,...,...,...,...,...,...,...
145,6.7,3.0,0.124931,0.055258,Virginica,Eric,0.0,0.0
146,6.3,2.5,5.000000,1.900000,Virginica,Sam,0.0,0.0
147,6.5,3.0,5.200000,2.000000,Virginica,Sam,0.0,0.0
148,6.2,3.4,0.129736,0.055258,Virginica,Eric,1.0,0.0


In [21]:
df

,sepal_length,sepal_width,petal_length,petal_width,variety,collector,predicate_proba,predicate_wrong_pred
0,5.1,3.5,1.4,0.2,Setosa,Sam,0.0,0.0
1,4.9,3.0,1.4,0.2,Setosa,Sam,0.0,0.0
2,4.7,3.2,1.3,0.2,Setosa,Sam,0.0,0.0
3,4.6,3.1,1.5,0.2,Setosa,Sam,0.0,0.0
4,5.0,3.6,1.4,0.2,Setosa,Sam,0.0,0.0
...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Virginica,Sam,0.0,0.0
146,6.3,2.5,5.0,1.9,Virginica,Sam,0.0,0.0
147,6.5,3.0,5.2,2.0,Virginica,Sam,0.0,0.0
148,6.2,3.4,5.4,2.3,Virginica,Sam,0.0,0.0


In [ ]:
from deepchecks.tabular import Dataset
ds = Dataset(df)

from deepchecks.tabular.suites import data_integrity

# Run Suite:
integ_suite = data_integrity()
suite_result = integ_suite.run(ds)
# Note: the result can be saved as html using suite_result.save_as_html()
# or exported to json using suite_result.to_json()
suite_result.show()